# Settings

**Configurando Spark para a exploração nativa em arquivos do lake house**

In [3]:
%%configure -f
{
 'conf':{
    'spark.native.enabled': 'true',
    'spark.shuffle.manager': 'org.apache.spark.shuffle.sort.ColumnarShuffleManager'
 }
}

StatementMeta(, 3e2814fc-6d8f-42c0-b51c-24066adf4062, -1, Finished, Available, Finished)

## Libraries

In [20]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

StatementMeta(, 3e2814fc-6d8f-42c0-b51c-24066adf4062, 20, Finished, Available, Finished)

# Content Training

In [6]:
%%pyspark

# Primeiro criando um dataframe spark com spark.read.csv e visualizando 
# as 10 primeiras linhas com display() e .limit()
df = spark.read.csv('Files/Data/sales.csv',
header=True,
inferSchema=False
)

display(df.limit(10))

StatementMeta(, 3e2814fc-6d8f-42c0-b51c-24066adf4062, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d1769441-02b5-42f4-ae98-f72cb25982e2)

In [37]:
# Criando um esquema de estrutura de tabela, usado principalmente ao criar 
# novos dataframes a partir de dados em estruturas como listas, que não tem nome de coluna
# ou tipo definido. Porém nesse caso há outra usabilidade que é de acelerar o
# processo de leitura do arquivo, deixando o esquema das colunas claro, o spark
# não precisa inferir automaticamente

# Aqui usamos as funções de pyspark.sql.types para a criação do esquema
typeSchema = StructType([
    StructField('SalesOrderNumber', StringType()),
    StructField('SalesOrderLineNumber', StringType()),
    StructField('OrderDate', DateType()),
    StructField('CustomerName', StringType()),
    StructField('EmailAddress', StringType()),
    StructField('Item', StringType()),
    StructField('Quantity', IntegerType()),
    StructField('UnitPrice', FloatType()),
    StructField('TaxAmount', FloatType()),
])

df = spark.read.csv('Files/Data/sales_wout_headers.csv',
schema=typeSchema,
header=False
)

display(df.limit(10))

StatementMeta(, 3e2814fc-6d8f-42c0-b51c-24066adf4062, 37, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 22503d1c-9885-4034-80e4-e67cf415f4f4)

In [21]:
# testando a seleção de colunas nos dataframes spark
display(df.select(['SalesOrderNumber', 'OrderDate']))

StatementMeta(, 3e2814fc-6d8f-42c0-b51c-24066adf4062, 21, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 410d8d05-14bd-468b-abe3-f8b34fb97003)

In [45]:
# testando a substituição de valores

# Substituindo os valores de Quantity para valores aleatórios entre 1 e 5 usando .withColumn
# para criar uma nova coluna, florr(rand() * 4) + 1, rand() gera um numero aleatório entre
# 0 e 1, após isso é multiplicado por 4, o que faz com que o calculo nunca passe de 4
# e que mesmo que chegue a valores mínimos como 0,0004 ou 0 ainda é adicionado +1
# no final da conta, que o deixa entre 1 e 5

df = df.withColumn('Quantity', floor(rand() * 4) + 1)

StatementMeta(, 3e2814fc-6d8f-42c0-b51c-24066adf4062, 45, Finished, Available, Finished)

In [46]:
# Criando uma nova coluna de Preço total usando col() de pyspark.sql.functions
# para acessarmos os valores da estrutura 'Column' de um dataframe Spark

df = df.withColumn('TotalPrice', col('UnitPrice') * col('Quantity'))
display(df.select(['Quantity', 'UnitPrice', 'TotalPrice']))

StatementMeta(, 3e2814fc-6d8f-42c0-b51c-24066adf4062, 46, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ff8f4985-8c47-41f0-80d2-bb3428c7ddf5)

In [36]:
# Testando agrupamentos usando .groupBy, .sum() e .orderBy()
display(df.groupBy('OrderDate').sum('TotalPrice').orderBy('OrderDate'))

StatementMeta(, 3e2814fc-6d8f-42c0-b51c-24066adf4062, 36, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fee8fbe7-c9e1-4c11-81cb-9b4549809eb9)

In [ ]:
top10prod = df.groupBy('Item').count().orderBy('count', ascending=False).limit(10)

In [43]:
display(top10prod)

StatementMeta(, 3e2814fc-6d8f-42c0-b51c-24066adf4062, 43, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2b71db91-1896-4c7c-b574-81c24500b748)

In [53]:
# Testando a sintaxe em cadeia e filtros usando .where()
display(
df \
.where(df['Item'] == 'Water Bottle - 30 oz.') \
.groupBy('OrderDate').sum('TotalPrice') \
.orderBy('OrderDate')
)

StatementMeta(, 3e2814fc-6d8f-42c0-b51c-24066adf4062, 53, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3f1ca1be-b63f-48fd-94e9-7c2647c561ca)

In [59]:
# Criando uma tabela temporária, que ficara disponivel apenas enquanto essa seção estiver
# ativa
df.createOrReplaceTempView('Sales_temp')

StatementMeta(, 3e2814fc-6d8f-42c0-b51c-24066adf4062, 59, Finished, Available, Finished)

In [63]:
# Criando uma tabela gerenciada, onde os metadados são salvos no metastore do spark
df.write.format('delta').saveAsTable('Sales_persistent')

StatementMeta(, 3e2814fc-6d8f-42c0-b51c-24066adf4062, 63, Finished, Available, Finished)

In [55]:
%%sql
--- Visualizando a tabela temporária com T-SQL

SELECT *
FROM Sales_temp

StatementMeta(, 3e2814fc-6d8f-42c0-b51c-24066adf4062, 55, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 10 fields>

In [62]:
%%sql

--- Criando consultas simples com a tabela temporária

SELECT OrderDate, SUM(Quantity * UnitPrice) AS TotalPrice
FROM Sales_temp
GROUP BY OrderDate
ORDER BY OrderDate ASC

StatementMeta(, 3e2814fc-6d8f-42c0-b51c-24066adf4062, 62, Finished, Available, Finished)

<Spark SQL result set with 914 rows and 2 fields>